<a href="https://colab.research.google.com/github/davidkorea/google_colaboratory/blob/master/keras_hongloumeng_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN, LSTM, GRU
from keras.optimizers import RMSprop
from keras.initializers import RandomNormal, Identity
from keras.datasets import mnist
from keras.utils import np_utils
import numpy as np
import random

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

In [23]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [24]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17108043207795042711, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18273111337176619606
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1910072626263280121
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281553818
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1531207984235154809
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [25]:
import os
os.listdir()

['.config', 'sample_data', 'hongloumeng.txt']

In [26]:
!ls

hongloumeng.txt  sample_data


In [27]:
with open('./hongloumeng.txt','r') as f:
    text = f.read()
    print('corpus length: ',len(text))
    print(text[100:200])
    text = text[:70000]

corpus length:  855343
曰"甄士隐"云云．但书中所记何事何人？自又云：“今风尘碌碌，一事无成，忽念及当日所有之女子，一一细考较去，觉其行止见识，皆出于我之上．何我堂堂须眉，诚不若彼裙钗哉？实愧则有余，悔又无益之大无可如何之日


In [28]:
chars = sorted(list(set(text)))
print('total chars: ',len(chars))
char_idx = dict((c,i) for i,c in enumerate(chars))
idx_char = dict((i,c) for i,c in enumerate(chars))

total chars:  2730


In [29]:
maxlen = 40
step = 3
sentences = [] # samples
next_chars = [] # labels
for i in range(0, len(text)-maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('num of total samples/sentences: ',len(sentences))

num of total samples/sentences:  23320


np.zeros( shape=(50000, 40, 4555)) maxmum = 50,000

In [0]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [0]:
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_idx[char]] = 1
  y[i, char_idx[next_chars[i]]] = 1

In [0]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
def sample(preds, diversity=1.0):
    preds = np.asarray(preds).astype(np.float64)
    preds = np.log(preds) / diversity
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(model, epoch):
    print()
    print('----- Generating text after Epoch: %d -----' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity: -----', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '" -----')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_idx[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = idx_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
        print()
        print(generated)
        print()

In [0]:
epochs = 60
# generator = Generator(128)
for epoch in range(epochs):
    model.fit(x, y, batch_size=128, epochs=1)
#     model.fit_generator(generator, steps_per_epoch=len(sentences)/128, epochs=1)
    on_epoch_end(model, epoch)

Epoch 1/1
23320/23320 [==============================] - 35s 1ms/step - loss: 5.2456

----- Generating text after Epoch: 0 -----
----- diversity: ----- 0.2
----- Generating with seed: "事要早走一步．贾瑞不敢强他，只得随他去了．这里茗烟先一把揪住金荣，问道：“我们у" -----

事要早走一步．贾瑞不敢强他，只得随他去了．这里茗烟先一把揪住金荣，问道：“我们у这里来，我们的这里的，不知的，不知道：“这里里，不知他们的，不知他们们的，也不知他们的，我们这里里的，我们的，不知他，不知他们的，不知道：“这里子们的，也不知道：“这里们们的，我们的这里的里，我们的，我们的去了．这里不知，便不知，我们的这里，不知道：“这里去了，不知的，不知道：“这里不知，不知他们的来，只有一个个个个个个子的，不知不知，只是一个个个个人来，我不知，这里的道：“这里这里的，不知道：“这里们这里的，不知这里，不知他这里，不知他们的，我们的道：“这里子的，不知的，不知他们，我们这里的，不知的，不知不知，这里不知，我们这里里来，不知道：“这里们们的这里的，不知他们的，不知他们们的了，我们的这里的来，不知道：“这里里的这里的的，我们这里的，不知他，不知道：“这里这里子们的，不知他们的，我们的这里的里的，我们的这里的，不知他们的，不知他们的，我们这里里来，我们这里的，不知他，不知道：“

----- diversity: ----- 0.5
----- Generating with seed: "事要早走一步．贾瑞不敢强他，只得随他去了．这里茗烟先一把揪住金荣，问道：“我们у" -----

事要早走一步．贾瑞不敢强他，只得随他去了．这里茗烟先一把揪住金荣，问道：“我们у家是这里里来，钗又里他们的的道：“这里相这里了．不知道：“这里里里，不知不过不来，我们的是一家，因有一个的子，不知有一里，我只里一个的，都不知道：“这里们这里子的不来，我们这里的家的不去，便说：“家们这里的了，若你一个好了。”宝玉道：“这日里子们们这里你们的看来，又有一个爷，不知不能说：“这里这里处子，我我们这里去了，我们的的好，也不知有这里子来，不知他，